In [26]:
import numpy as np
import pandas as pd

# Tìm nghiệm của đa thức $f(x) = \sum\limits_{i=0}^n a_i x^i = 0$


In [27]:
def polynomial(coefficients, x):
    return sum(c * x**i for i, c in enumerate(coefficients))

def polynomial_derivative(coefficients, x):
    return sum(i * c * x**(i-1) for i, c in enumerate(coefficients) if i > 0)

def polynomial_second_derivative(coefficients, x):
    return sum(i * (i-1) * c * x**(i-2) for i, c in enumerate(coefficients) if i > 1)

# 1. Bán kính nghiệm

* Bán kính phức: $R = 1 + \dfrac{\max |a_i|}{|a_n|}$

* Cận trên của các nghiệm dương: Với $a_n > 0$, $a_k < 0$ (k lớn nhất có thể): $R_{upper} = 1 + \sqrt[n-k] {\dfrac{\max\limits_{a_i < 0} |a_i|}{|a_n|}}$

* Cận dưới của các nghiệm âm $f(x) = 0$ $\Leftrightarrow$ Cận trên của các nghiệm dương $f(-x) = 0$ 

In [28]:
def complex_radius(coefficients):
    R = 1 + max(abs(coeff) for coeff in coefficients) / abs(coefficients[-1])
    return R

def real_radius(coefficients):
    n = len(coefficients) - 1
    negative_coefficients = [(i, abs(x)) for i, x in enumerate(coefficients) if x<0]
    k = (max(negative_coefficients, key=lambda item: item[0]))[0]
    ratio = (max(negative_coefficients, key=lambda item: item[1]))[1] / abs(coefficients[-1])
    
    R = 1 + ratio**(1/(n-k)) if n!=k else 0
    return R

In [29]:
# Example usage
coefficients = [-15, 0, 3, 0, 3, 1] # Coefficients of the polynomial -15 + 3x^2 - ax^4 + x^5
temp = [x if i%2==0 else -x for i, x in enumerate(coefficients)]
if (temp[-1] < 0):
    revert_coefficients = [-i for i in temp]
else:
    revert_coefficients = temp

radius = complex_radius(coefficients)
upper_radius = real_radius(coefficients)
lower_radius = -real_radius(revert_coefficients)

print(f"Complex radius for roots: {-radius} -> {radius}")
print("Upper radius for real-value roots:", upper_radius)
print("Lower radius for real-value roots:", lower_radius)

Complex radius for roots: -16.0 -> 16.0
Upper radius for real-value roots: 2.718771927587479
Lower radius for real-value roots: -4.0


# 2. Tìm các cực trị của đa thức

Sử dụng phương pháp Gradient Descent để tìm tất cả các cực trị của hàm đa thức dựa trên các hệ số đầu vào.

## Thuật toán

* B1. Nguyên lý hoạt động

    Thuật toán sử dụng phương pháp Gradient Descent/Ascent với tốc độ học thích ứng để tìm các cực trị địa phương của đa thức. Quá trình được thực hiện như sau:

    * a. Khởi tạo
        - Tạo nhiều điểm khởi đầu phân bố đều trong khoảng `[lower_bound, upper_bound]`
        - Mỗi điểm khởi đầu được sử dụng để tìm cả cực đại và cực tiểu
        - Khởi tạo tốc độ học ban đầu (`learning_rate` $=\alpha$)

    * b. Quá trình tìm kiếm
        - **Gradient Descent** (tìm cực tiểu):
            + Di chuyển ngược hướng gradient
            + $x_{k+1} = x_k - \alpha \cdot f'(x_k)$
        - **Gradient Ascent** (tìm cực đại):
            + Di chuyển cùng hướng gradient
            + $x_{k+1} = x_k + \alpha \cdot f'(x_k)$

    * c. Cơ chế tốc độ học thích ứng
        - Kiểm tra hướng di chuyển:
            + Nếu tìm cực đại mà giá trị giảm → sai hướng
            + Nếu tìm cực tiểu mà giá trị tăng → sai hướng
        - Khi di chuyển sai hướng:
            + Giảm tốc độ học một nửa: $\alpha_1 = \alpha \cdot 0.5$
            + Thử lại bước di chuyển với tốc độ học mới

* B2. Điều kiện dừng và xác nhận cực trị

    * a. Điều kiện dừng
        - Khoảng cách giữa hai bước liên tiếp nhỏ hơn ngưỡng: `|x_new - x| < tolerance`
        - Hoặc đạt số bước lặp tối đa: `max_iterations`

    * b. Xác nhận cực trị
        - Sử dụng đạo hàm bậc hai để xác nhận:
            + Cực tiểu: đạo hàm bậc hai > 0
            + Cực đại: đạo hàm bậc hai < 0

## Khoảng cách lý nghiệm:

- Xếp các `minimum`,`maximum`, `lower_radius`, `upper_radius` thành 1 mảng theo thứ tự tăng dần
- Nếu 2 phần tử liên tiếp có giá trị ngược dấu nhau -> *Khoảng cách ly nghiệm*

## Áp dụng

In [30]:
def find_extrema(coefficients, lower_bound, upper_bound, learning_rate=0.01, max_iterations=1000, tolerance=1e-6, num_starting_points=10):
    """
    Find local minima and maxima using gradient descent/ascent with adaptive learning rate
    """
    starting_points = np.linspace(lower_bound, upper_bound, num_starting_points)
    extrema = {'minima': [], 'maxima': []}
    
    def gradient_search(x0, direction=1):  # direction: 1 for maxima, -1 for minima
        x = x0
        current_lr = learning_rate
        prev_value = polynomial(coefficients, x)
        
        for _ in range(max_iterations):
            gradient = polynomial_derivative(coefficients, x)
            x_new = x + direction * current_lr * gradient
            
            # Stay within bounds
            x_new = np.clip(x_new, lower_bound, upper_bound)
            
            # Check if we're moving in the wrong direction
            current_value = polynomial(coefficients, x_new)
            if (direction == 1 and current_value < prev_value) or \
               (direction == -1 and current_value > prev_value):
                # We went too far, reduce learning rate and try again
                current_lr *= 0.5
                continue
                
            # Update position if improvement is made
            if abs(x_new - x) < tolerance:
                # Check if it's actually an extremum using second derivative
                second_deriv = polynomial_second_derivative(coefficients, x_new)
                if (direction == -1 and second_deriv > 0) or (direction == 1 and second_deriv < 0):
                    return x_new
                return None
                
            x = x_new
            prev_value = current_value
            
        return None

    # Rest of the function remains the same
    for x0 in starting_points:
        # Find minimum
        min_point = gradient_search(x0, direction=-1)
        if min_point is not None:
            if not any(abs(min_point - x) < tolerance for x in extrema['minima']):
                extrema['minima'].append(min_point)
        
        # Find maximum
        max_point = gradient_search(x0, direction=1)
        if max_point is not None:
            if not any(abs(max_point - x) < tolerance for x in extrema['maxima']):
                extrema['maxima'].append(max_point)
    
    # Sort the results
    extrema['minima'].sort()
    extrema['maxima'].sort()
    return extrema

In [31]:
# Test the function
extrema = find_extrema(coefficients, lower_radius, upper_radius)
print("\nLocal minima:", [f"{x}" for x in extrema['minima']])
print("Local maxima:", [f"{x}" for x in extrema['maxima']])

# Optionally, print function values at extrema
print("\nFunction values at minima:", [f"{polynomial(coefficients, x)}" for x in extrema['minima']])
print("Function values at maxima:", [f"{polynomial(coefficients, x)}" for x in extrema['maxima']])


Local minima: ['-1.5090528188950435e-05', '1.4929626854160653e-05']
Local maxima: ['-2.5802435861430424']

Function values at minima: ['-14.999999999316827', '-14.999999999331319']
Function values at maxima: ['23.578420506959375']
